# Amplifierd API - Sessions & Messages

This notebook demonstrates session management and messaging capabilities.

## Overview

Sessions represent conversational contexts for LLM interactions. Each session:
- Has a unique ID
- Uses a specific profile
- Maintains a message transcript
- Has explicit lifecycle states (CREATED → ACTIVE → COMPLETED/FAILED/TERMINATED)
- Operates within an amplified directory context

## Setup

Import libraries and configure the client:

In [ ]:
import json

import requests

BASE_URL = "http://127.0.0.1:8420"
API_BASE = f"{BASE_URL}/api/v1"


def print_response(response: requests.Response, title: str = "") -> None:
    """Print formatted HTTP response."""
    if title:
        print(f"\n{'=' * 60}")
        print(f"{title}")
        print(f"{'=' * 60}")
    print(f"Status: {response.status_code} {response.reason}")
    if response.content:
        try:
            print(json.dumps(response.json(), indent=2))
        except json.JSONDecodeError:
            print(response.text)


print("✓ Setup complete")

## Creating a Session

Create a new session with a profile. The session will be created in **CREATED** state and must be explicitly started before use.

### Request Parameters

- **amplified_dir** (optional): Relative path to amplified directory, defaults to "."
- **profile_name** (optional): Profile to use. If not provided, uses directory's default_profile
- **parent_session_id** (optional): Parent session ID for sub-sessions
- **settings_overrides** (optional): Settings to override profile defaults

In [ ]:
# Create a session with default profile
# Note: amplified_dir="." uses the root amplified directory
create_data = {
    "amplified_dir": ".",  # Current amplified directory
    "profile_name": "foundation/base",  # Explicitly specify profile
}

response = requests.post(f"{API_BASE}/sessions/", json=create_data)
print_response(response, "CREATE SESSION")

# Save session ID for later use
if response.ok:
    session_data = response.json()
    session_id = session_data["sessionId"]
    print(f"\n✓ Created session: {session_id}")
    print(f"   Status: {session_data['status']}")
    print(f"   Profile: {session_data['profileName']}")
else:
    session_id = None
    print("\n✗ Failed to create session")

## Session Lifecycle

Sessions have an explicit lifecycle:

1. **CREATED** - Initial state after creation
2. **ACTIVE** - Session is running and can receive messages
3. **COMPLETED** - Session finished successfully
4. **FAILED** - Session encountered an error
5. **TERMINATED** - Session was terminated early

### Starting a Session

Before you can send messages to a session, you must start it:

In [ ]:
if session_id:
    response = requests.post(f"{API_BASE}/sessions/{session_id}/start")
    print_response(response, f"START SESSION: {session_id}")
    
    if response.status_code == 204:
        print("\n✓ Session started successfully")
        print("   Session is now ACTIVE and ready for messages")
else:
    print("No session ID available. Create a session first.")

## Listing Sessions

Get all available sessions:

In [ ]:
response = requests.get(f"{API_BASE}/sessions/")
print_response(response, "LIST SESSIONS")

if response.ok:
    sessions = response.json()
    print(f"\n✓ Found {len(sessions)} session(s)")
    
    for session in sessions:
        print(f"   • {session['sessionId']}: {session['status']} (profile: {session['profileName']})")

## Getting Session Details

Retrieve information about a specific session:

In [ ]:
if session_id:
    response = requests.get(f"{API_BASE}/sessions/{session_id}")
    print_response(response, f"GET SESSION: {session_id}")
else:
    print("No session ID available. Create a session first.")

## Adding Messages to Session

Add a message to the session transcript. This does NOT execute the message with LLM - it just adds it to the transcript.

For executing messages with LLM, you would use:
- `POST /sessions/{id}/send-message` (requires SSE stream connection)
- `GET /sessions/{id}/stream` (SSE endpoint for receiving responses)

In [ ]:
if session_id:
    message_data = {
        "content": "Hello, amplifierd! This is a test message.",
    }

    response = requests.post(f"{API_BASE}/sessions/{session_id}/messages", json=message_data)
    print_response(response, "ADD MESSAGE TO TRANSCRIPT")
    
    if response.ok:
        print("\n✓ Message added to transcript")
        print("   Note: This adds to transcript but does NOT execute with LLM")
        print("   For LLM execution, use /send-message with SSE stream")
else:
    print("No session ID available. Create a session first.")

## Retrieving Transcript

Get the full message transcript for a session:

In [ ]:
if session_id:
    response = requests.get(f"{API_BASE}/sessions/{session_id}/transcript")
    print_response(response, "GET TRANSCRIPT")

    if response.ok:
        messages = response.json()
        print(f"\n✓ Retrieved {len(messages)} message(s)")
        for msg in messages:
            content_preview = msg['content'][:50] + "..." if len(msg['content']) > 50 else msg['content']
            print(f"   - [{msg['role']}] {content_preview}")
else:
    print("No session ID available. Create a session first.")

## Adding Multiple Messages

Simulate a conversation by adding multiple messages:

In [ ]:
if session_id:
    messages_to_send = [
        {"content": "What are amplifier profiles?"},
        {"content": "How do I activate a profile?"},
        {"content": "Can I list all available modules?"},
    ]

    for msg_data in messages_to_send:
        response = requests.post(f"{API_BASE}/sessions/{session_id}/messages", json=msg_data)
        if response.ok:
            print(f"✓ Added: {msg_data['content']}")
        else:
            print(f"✗ Failed: {msg_data['content']}")

    # Get updated transcript
    response = requests.get(f"{API_BASE}/sessions/{session_id}/transcript")
    if response.ok:
        messages = response.json()
        print(f"\n✓ Total messages in session: {len(messages)}")
else:
    print("No session ID available. Create a session first.")

## Completing a Session

When work is done, complete the session to transition it to COMPLETED state:

In [ ]:
if session_id:
    response = requests.post(f"{API_BASE}/sessions/{session_id}/complete")
    print_response(response, f"COMPLETE SESSION: {session_id}")
    
    if response.status_code == 204:
        print("\n✓ Session completed successfully")
        print("   Status: COMPLETED")
else:
    print("No session ID available. Create a session first.")

## Error Handling: Non-Existent Session

Attempting to access a session that doesn't exist:

In [ ]:
fake_session_id = "session_nonexistent"
response = requests.get(f"{API_BASE}/sessions/{fake_session_id}")
print_response(response, "GET NON-EXISTENT SESSION")

if response.status_code == 404:
    print("\n✓ Correctly returns 404 for non-existent session")

## Deleting a Session

Clean up by deleting the session:

In [ ]:
if session_id:
    response = requests.delete(f"{API_BASE}/sessions/{session_id}")
    print_response(response, f"DELETE SESSION: {session_id}")

    if response.status_code == 204:
        print("\n✓ Session deleted successfully")

        # Verify deletion
        response = requests.get(f"{API_BASE}/sessions/{session_id}")
        if response.status_code == 404:
            print("✓ Confirmed session no longer exists")
else:
    print("No session ID available.")

## Complete Session Lifecycle Example

Full workflow from creation to deletion with proper lifecycle management:

In [ ]:
def session_lifecycle_example():
    """Demonstrate complete session lifecycle."""

    # 1. Create session
    print("1. Creating session...")
    response = requests.post(
        f"{API_BASE}/sessions/", 
        json={
            "amplified_dir": ".",
            "profile_name": "foundation/base"
        }
    )
    if not response.ok:
        print("✗ Failed to create session")
        return

    session_data = response.json()
    session_id = session_data["sessionId"]
    print(f"✓ Created: {session_id} (status: {session_data['status']})")

    # 2. Start session
    print("\n2. Starting session...")
    response = requests.post(f"{API_BASE}/sessions/{session_id}/start")
    if response.status_code == 204:
        print("✓ Session started (status: ACTIVE)")

    # 3. Add messages
    print("\n3. Adding messages...")
    for i in range(3):
        response = requests.post(
            f"{API_BASE}/sessions/{session_id}/messages", 
            json={"content": f"Message {i + 1}"}
        )
        if response.ok:
            print(f"✓ Added message {i + 1}")

    # 4. Retrieve transcript
    print("\n4. Retrieving transcript...")
    response = requests.get(f"{API_BASE}/sessions/{session_id}/transcript")
    if response.ok:
        messages = response.json()
        print(f"✓ Retrieved {len(messages)} messages")

    # 5. Complete session
    print("\n5. Completing session...")
    response = requests.post(f"{API_BASE}/sessions/{session_id}/complete")
    if response.status_code == 204:
        print("✓ Session completed (status: COMPLETED)")

    # 6. Delete session
    print("\n6. Deleting session...")
    response = requests.delete(f"{API_BASE}/sessions/{session_id}")
    if response.status_code == 204:
        print("✓ Deleted successfully")

    print("\n✓ Lifecycle complete")


session_lifecycle_example()

## Failing a Session

If a session encounters an error, you can mark it as FAILED with an error message:

In [ ]:
# Create and start a test session
response = requests.post(
    f"{API_BASE}/sessions/",
    json={"amplified_dir": ".", "profile_name": "foundation/base"}
)
if response.ok:
    test_session_id = response.json()["sessionId"]
    
    # Start it
    requests.post(f"{API_BASE}/sessions/{test_session_id}/start")
    
    # Fail it with an error message
    response = requests.post(
        f"{API_BASE}/sessions/{test_session_id}/fail",
        json={"error_message": "Simulated error for demonstration"}
    )
    print_response(response, "FAIL SESSION")
    
    if response.status_code == 204:
        print("\n✓ Session marked as FAILED")
        print("   Status: FAILED")
        print("   Error: 'Simulated error for demonstration'")
    
    # Clean up
    requests.delete(f"{API_BASE}/sessions/{test_session_id}")

## Terminating a Session

You can also terminate a session early:

In [ ]:
# Create and start a test session
response = requests.post(
    f"{API_BASE}/sessions/",
    json={"amplified_dir": ".", "profile_name": "foundation/base"}
)
if response.ok:
    test_session_id = response.json()["sessionId"]
    
    # Start it
    requests.post(f"{API_BASE}/sessions/{test_session_id}/start")
    
    # Terminate it
    response = requests.post(f"{API_BASE}/sessions/{test_session_id}/terminate")
    print_response(response, "TERMINATE SESSION")
    
    if response.status_code == 204:
        print("\n✓ Session terminated")
        print("   Status: TERMINATED")
    
    # Clean up
    requests.delete(f"{API_BASE}/sessions/{test_session_id}")

## Summary

You've learned:
- ✓ Creating sessions with amplified_dir and profile_name
- ✓ Session lifecycle states (CREATED → ACTIVE → COMPLETED/FAILED/TERMINATED)
- ✓ Starting sessions before use
- ✓ Listing and retrieving session details
- ✓ Adding messages to transcript (sync, no execution)
- ✓ Retrieving full transcript
- ✓ Completing, failing, or terminating sessions
- ✓ Deleting sessions
- ✓ Complete session lifecycle management

## Key Differences from Previous API

1. **New lifecycle**: Sessions now have explicit states (CREATED, ACTIVE, COMPLETED, FAILED, TERMINATED)
2. **Must start**: Sessions must be explicitly started with `POST /sessions/{id}/start` before use
3. **New parameters**: `amplified_dir` and `profile_name` replace old `profile` and `context`
4. **Transcript endpoint**: Use `/transcript` instead of `/messages` for GET requests
5. **Message execution**: `/messages` is sync (no execution), `/send-message` is async (with execution via SSE)
6. **No resume endpoint**: Sessions don't need resuming - they persist automatically

## API Endpoints Reference

| Method | Endpoint | Description |
|--------|----------|-------------|
| POST | `/api/v1/sessions/` | Create new session (status: CREATED) |
| GET | `/api/v1/sessions/` | List all sessions |
| GET | `/api/v1/sessions/{id}` | Get session details |
| POST | `/api/v1/sessions/{id}/start` | Start session (CREATED → ACTIVE) |
| POST | `/api/v1/sessions/{id}/complete` | Complete session (ACTIVE → COMPLETED) |
| POST | `/api/v1/sessions/{id}/fail` | Fail session (ACTIVE → FAILED) |
| POST | `/api/v1/sessions/{id}/terminate` | Terminate session (ACTIVE → TERMINATED) |
| DELETE | `/api/v1/sessions/{id}` | Delete session |
| POST | `/api/v1/sessions/{id}/messages` | Add message to transcript (sync) |
| POST | `/api/v1/sessions/{id}/send-message` | Send message for execution (async) |
| GET | `/api/v1/sessions/{id}/transcript` | Get full transcript |
| GET | `/api/v1/sessions/{id}/stream` | SSE stream for messages (for send-message) |

## Next Steps

Continue to other notebooks:
- **03-profile-management.ipynb** - Profile operations
- **04-collection-management.ipynb** - Collection management
- **05-module-management.ipynb** - Module configuration
- **06-streaming-sse.ipynb** - Server-Sent Events for streaming responses
- **07-session-lifecycle.ipynb** - Enhanced session lifecycle patterns
- **08-amplified-directories.ipynb** - Multi-directory context management